In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Function, Variable
from pathlib import Path
from itertools import groupby
import json
import glob
import pytesseract
import time

import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
input_dir = r"C:\Users\masan\Downloads\IDRBT Cheque Image Dataset\COCO\DIR\annotations"
train_img_dir = r"C:\Users\masan\Downloads\IDRBT Cheque Image Dataset\COCO\DIR\annotations"
test_img_dir = r"C:\Users\masan\Downloads\IDRBT Cheque Image Dataset\COCO\DIR\annotations\val"

WIDTH = 512
HEIGHT = 256
category_num = 5 + 1

ratio = 8

epoch_num = 50
batch_size = 4

device = "cuda:0"

In [19]:
# def make_onehot_vec(x):
#     vec = np.zeros(category_num)
#     vec[x] = 1
#     return vec

In [20]:
file = open(r'COCO\DIR\annotations\instances_train.json','r')
annotations_train = json.load(file)
train_images = [(x['file_name'],x['id'],x['height'],x['width']) for x in annotations_train['images']]
train_dict = {}
for x in annotations_train['annotations']:
    if x['image_id'] not in train_dict:
        train_dict[x['image_id']] = [[x, train_images[x['image_id']]]]
    else:
        train_dict[x['image_id']].append([x, train_images[x['image_id']]])
train_dict

{0: [[{'segmentation': [[321, 107, 321, 153, 617, 153, 617, 107]],
    'iscrowd': 0,
    'area': 13616,
    'image_id': 0,
    'bbox': [321, 107, 296, 46],
    'category_id': 2,
    'id': 1},
   ('10.jpg', 0, 1100, 2365)],
  [{'segmentation': [[777, 145, 777, 197, 1079, 197, 1079, 145]],
    'iscrowd': 0,
    'area': 15704,
    'image_id': 0,
    'bbox': [777, 145, 302, 52],
    'category_id': 3,
    'id': 2},
   ('10.jpg', 0, 1100, 2365)],
  [{'segmentation': [[557, 959, 557, 1067, 1761, 1067, 1761, 959]],
    'iscrowd': 0,
    'area': 130032,
    'image_id': 0,
    'bbox': [557, 959, 1204, 108],
    'category_id': 4,
    'id': 3},
   ('10.jpg', 0, 1100, 2365)],
  [{'segmentation': [[413, 509, 413, 613, 1221, 613, 1221, 509]],
    'iscrowd': 0,
    'area': 84032,
    'image_id': 0,
    'bbox': [413, 509, 808, 104],
    'category_id': 1,
    'id': 4},
   ('10.jpg', 0, 1100, 2365)]],
 1: [[{'segmentation': [[231, 36, 231, 102, 659, 102, 659, 36]],
    'iscrowd': 0,
    'area': 28248,
  

In [21]:

file = open(r'C:\Users\masan\Downloads\IDRBT Cheque Image Dataset\COCO\DIR\annotations\instances_val.json','r')
annotations_val = json.load(file)
val_images = [(x['file_name'],x['id'],x['height'],x['width']) for x in annotations_val['images']]
val_dict = {}
for x in annotations_val['annotations']:
    if x['image_id'] not in val_dict:
        val_dict[x['image_id']] = [[x, val_images[x['image_id']]]]
    else:
        val_dict[x['image_id']].append([x, val_images[x['image_id']]])

In [22]:
seg_img = np.full((10,20), category_num-1, dtype=np.int32)
seg_img

array([[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]])

In [23]:
def make_mask_img(info):
    seg_width = info[0][1][3]
    seg_height = info[0][1][2]
    seg_img = np.full((seg_height,seg_width), category_num-1, dtype=np.int32)
    
    image_info = info[0][1]
    for i, x in enumerate(info):
        mask_img_seg = x[0]['bbox']

        x1 = mask_img_seg[1]
        y1 = mask_img_seg[0]
        h = mask_img_seg[2]
        w = mask_img_seg[3]

        seg_img[x1:x1+w, y1:y1+h] = x[0]['category_id']
        #seg_img[x1:x1+w, y1:y1+h] = x[0]['category_id']
        
    seg_img = seg_img.reshape((seg_height, seg_width), order='F')
    seg_img = cv2.resize(seg_img, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    
    return seg_img

#     for encoded_pixels, class_id in zip(segment_df["EncodedPixels"].values, segment_df["ClassId"].values):
#         pixel_list = list(map(int, encoded_pixels.split(" ")))
#         for i in range(0, len(pixel_list), 2):
#             start_index = pixel_list[i] - 1
#             index_len = pixel_list[i+1] - 1
#             seg_img[start_index:start_index+index_len] = int(class_id.split("_")[0])
#     seg_img = seg_img.reshape((seg_height, seg_width), order='F')
#     seg_img = cv2.resize(seg_img, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
#     return seg_img

In [24]:
# count = 0
# for x in train_dict:
#     cv2.imwrite(f'{time.time()}.jpg',make_mask_img(train_dict[x]))
#     count+=1

In [25]:
def train_generator(train_data, batch_size):
#     img_ind_num = df.groupby("ImageId")["ClassId"].count()
#     index = df.index.values[0]
    trn_images = []
    seg_images = []
    for i, (img_id, data) in enumerate(train_data):
        img = cv2.imread(os.path.join(train_img_dir,"train",data[0][1][0]))
        img = cv2.resize(img, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
        seg_img = make_mask_img(data)

        img = img.transpose((2, 0, 1))
        #seg_img = seg_img.transpose((2, 0, 1))
        
        trn_images.append(img)
        seg_images.append(seg_img)
        if((i+1) % batch_size == 0):
            yield np.array(trn_images, dtype=np.float32) / 255, np.array(seg_images, dtype=np.int32)
            trn_images = []
            seg_images = []
    if(len(trn_images) != 0):
        yield np.array(trn_images, dtype=np.float32) / 255, np.array(seg_images, dtype=np.int32)

In [26]:
def test_generator(location):
    img_names = glob.glob(f"{location}/*.jpg")
    for img_name in img_names:
        img = cv2.imread(img_name)
        img = cv2.resize(img, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
        # HWC -> CHW
        img = img.transpose((2, 0, 1))
        yield img_name, np.asarray([img], dtype=np.float32) / 255

In [27]:
def encode(input_string):
    return [(len(list(g)), k) for k,g in groupby(input_string)]

def run_length(label_vec):
    encode_list = encode(label_vec)
    index = 1
    class_dict = {}
    for i in encode_list:
        if i[1] != category_num-1:
            if i[1] not in class_dict.keys():
                class_dict[i[1]] = []
            class_dict[i[1]] = class_dict[i[1]] + [index, i[0]]
        index += i[0]
    return class_dict

In [38]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        #self.resnet = torchvision.models.resnet50(pretrained=True, progress=True)
        #self.resnet.fc = nn.Linear(4, 2048)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        #x1 = self.resnet(x)
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

In [39]:
print (category_num)
net = UNet(n_channels=3, n_classes=category_num).to(device)

optimizer = optim.SGD(
    net.parameters(),
    lr=0.05,
    momentum=0.99,
    weight_decay=0.0005,
    
)

criterion = nn.CrossEntropyLoss()

6


In [40]:
train_loss = []
valid_loss = []
train_end = int(len(train_dict)*0.8)
items = list(train_dict.items())
np.random.shuffle(items)

for epoch in range(epoch_num):
    epoch_trn_loss = 0
    train_len = 0
    net.train()
    for iteration, (X_trn, Y_trn) in enumerate(tqdm(train_generator(items[:train_end], batch_size))):
        X = torch.tensor(X_trn, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_trn, dtype=torch.long).to(device)
        train_len += len(X)
        
        #Y_flat = Y.view(-1)
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_trn_loss += loss.item()
        
        if iteration % 100 == 0:
            print("train loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_trn_loss/(iteration+1)))
        
    train_loss.append(epoch_trn_loss/(iteration+1))
    print("train {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, train_loss[-1]))
    
    epoch_val_loss = 0
    val_len = 0
    net.eval()
    for iteration, (X_val, Y_val) in enumerate(tqdm(train_generator(items[train_end:], batch_size))):
        X = torch.tensor(X_val, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_val, dtype=torch.long).to(device)
        val_len += len(X)
        
        #Y_flat = Y.view(-1)
        
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        epoch_val_loss += loss.item()
        
        if iteration % 100 == 0:
            print("valid loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_val_loss/(iteration+1)))
        
    valid_loss.append(epoch_val_loss/(iteration+1))
    print("valid {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, valid_loss[-1]))

RuntimeError: size mismatch, m1: [4 x 2048], m2: [4 x 2048] at C:/w/1/s/tmp_conda_3.7_100118/conda/conda-bld/pytorch_1579082551706/work/aten/src\THC/generic/THCTensorMathBlas.cu:290

In [ ]:
torch.save(net.state_dict(), r'C:\Users\masan\Downloads\IDRBT Cheque Image Dataset\models\newdata-50-4-0.01.pth')

In [21]:
def bbox_points(mask_prob,class_id):
    result = np.where(mask_prob[:,:]==class_id)
    x0 = min(result[0])
    x1 = max(result[0])
    y0 = min(result[1])
    y1 = max(result[1])
    
    return (y0,x0),(y1,x1)

In [30]:
import imutils,cv2
from skimage.segmentation import clear_border
from imutils import contours
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

sub_list = []
net.eval()
count = 0
for img_name, img in test_generator('single_inference'):
    X = torch.tensor(img, dtype=torch.float32).to(device)
    mask_pred = net(X)
    mask_pred1 = mask_pred.cpu().detach().numpy()
    mask_prob = np.argmax(mask_pred1, axis=1)
    mask_prob1 = mask_prob.ravel(order='F')
    class_dict = run_length(mask_prob1)
    print (mask_prob.shape)
    
    img = cv2.imread(img_name)
    img = cv2.resize(img, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
    
    mask_prob = mask_prob.transpose((1, 2, 0))
    print (np.unique(mask_prob))
    
    mask = np.zeros((HEIGHT, WIDTH),np.float32)
    indices = np.where(mask_prob!=5)
    mask[indices[0],indices[1]] = 255
    

    
    color = (255, 0, 0) 
    thickness = 2
    
    pred_classes = np.unique(mask_prob)
    if 0 in pred_classes:
        class_0 = bbox_points(mask_prob,0)
        img = cv2.rectangle(img, class_0[0], class_0[1], color, thickness) 
    
    if 1 in pred_classes:
        class_1 = bbox_points(mask_prob,1)
        img = cv2.rectangle(img, class_1[0], class_1[1], color, thickness)
    
    if 2 in pred_classes:
        class_2 = bbox_points(mask_prob,2)
        img = cv2.rectangle(img, class_2[0], class_2[1], color, thickness)
    
    if 3 in pred_classes:
        class_3 = bbox_points(mask_prob,3)
        img = cv2.rectangle(img, class_3[0], class_3[1], color, thickness)
    
    if 4 in pred_classes:
        class_4 = bbox_points(mask_prob,4)
        img =  cv2.rectangle(img, class_4[0], class_4[1], color, thickness)
    
    cv2.imwrite(f"C:\\Users\\masan\\Downloads\\IDRBT Cheque Image Dataset\\{count}.jpg",img)
    count+=1


(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[1 4 5]
(1, 256, 512)
[1 4 5]
(1, 256, 512)
[1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]
(1, 256, 512)
[0 1 2 4 5]


In [ ]:
def augment_image(img,mask):
    augs = []
    
    rot = np.random.randint(-15,15)
    rotated_img = rotate(img, angle=rot, mode = 'wrap')
    rotated_mask = rotate(mask, angle=rot, mode = 'wrap')
    
    augs.append((rotated_img,rotated_mask))
    
    sigma=0.155
    noisyRandom = random_noise(img,var=sigma**2)
    
    augs.append((noisyRandom,mask))
    
    blurred = gaussian(img,sigma=1,multichannel=True)
    augs.append((blurred,mask))
